# A Second Example

In [26]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import re

/pbs/throng/training/universite-hiver/rag-envs/rag2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
# Load models
model_general = SentenceTransformer('all-MiniLM-L6-v2')
tokenizer_sci = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model_sci = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')

In [14]:
data = pd.read_csv('data/arxiv_data.csv')

In [15]:
len(data)

51774

In [16]:
data.head()

,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"


In [22]:
data.terms.apply(eval).explode().value_counts().head(10)

terms
cs.CV      30413
cs.LG      29067
stat.ML    15578
cs.AI       7944
eess.IV     2484
cs.RO       1896
cs.CL       1620
cs.NE       1296
cs.CR        717
cs.SI        678
Name: count, dtype: int64

In [44]:
data2 = data[data['terms'].str.contains('cs.CL')][['titles', 'summaries']].reset_index(drop=True)

In [24]:
len(data2)

1620

In [28]:
def embed_with_scibert(texts, batch_size=16):
    embeddings = []
    model_sci.eval()
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size)):
            batch = texts[i:i+batch_size]
            encoded = tokenizer_sci(batch, padding=True, truncation=True, return_tensors='pt')
            output = model_sci(**encoded)
            mean_pooling = output.last_hidden_state.mean(dim=1)
            embeddings.append(mean_pooling)
    return torch.cat(embeddings).cpu().numpy()

In [30]:
# Embed abstracts
summaries = data2['summaries'].tolist()
embeddings_general = model_general.encode(summaries, convert_to_numpy=True)
embeddings_sci = embed_with_scibert(summaries)

100%|██████████| 102/102 [00:50<00:00,  2.01it/s]


In [31]:
def count_keyword_matches(text, query):
    query_words = set(re.findall(r'\w+', query.lower()))
    text_words = set(re.findall(r'\w+', text.lower()))
    return len(query_words.intersection(text_words))

In [58]:
def get_top_k_chunks(data, query, mode="keywords", top_k=5):
    query_emb_sci = embed_with_scibert([query])[0].reshape(1, -1)
    query_emb_general = model_general.encode([query])[0].reshape(1, -1)

    results = []

    for i in range(len(data)):
        summary = data.loc[i, 'summaries']
        title = data.loc[i, 'titles']
        keyword_hits = count_keyword_matches(summary, query)

        sim_sci = cosine_similarity(query_emb_sci, embeddings_sci[i].reshape(1, -1))[0][0]
        sim_gen = cosine_similarity(query_emb_general, embeddings_general[i].reshape(1, -1))[0][0]

        results.append({
            "title": title,
            "summary": summary,
            "keyword_matches": keyword_hits,
            "sim_sciBERT": sim_sci,
            "sim_miniLM": sim_gen
        })

    if mode == "keywords":
        ranked = sorted(results, key=lambda x: x['keyword_matches'], reverse=True)
    elif mode == "scibert":
        ranked = sorted(results, key=lambda x: x['sim_sciBERT'], reverse=True)
    elif mode == "minilm":
        ranked = sorted(results, key=lambda x: x['sim_miniLM'], reverse=True)
    else:
        raise ValueError("Mode must be one of: 'keywords', 'scibert', 'minilm'")

    return ranked[:top_k]

In [51]:
query = "Approaches combining knowledge graphs and text"

In [97]:
answer = client.chat(model, messages=[
        {
            'role': 'user',
            'content': f"""Approaches combining knowledge graphs and text"""
        },
    ])

In [104]:
print(answer.message.content)

## Combining Knowledge Graphs (KGs) and Text: A Quick‑Start Guide

| # | Category | Typical Methods | Core Idea | Representative Papers |
|---|----------|-----------------|-----------|-----------------------|
| 1 | **KG Embedding + Textual Context** | Joint learning of KG triples and document sentences | Encode entities/relations **and** sentences into a shared vector space | *KG-BERT* (2019), *K-BERT* (2020), *ERNIE* (2020) |
| 2 | **Graph Neural Networks (GNNs) + Text** | Use a GNN to propagate information across entities while a Transformer processes text | Combine local graph structure with global language context | *Text2KG* (2019), *KEPLER* (2020) |
| 3 | **Knowledge‑Enhanced Language Models** | Augment pre‑training or fine‑tuning with KG facts | Use KG to guide attention or add a knowledge head | *K-Adapter* (2021), *K-BERT* (2020), *K-GPT* (2021) |
| 4 | **KG‑Guided Retrieval + Text Generation** | Retrieve relevant KG sub‑graphs before generating or classifying | Provide a “mem

## Combining Knowledge Graphs (KGs) and Text: A Quick‑Start Guide

| # | Category | Typical Methods | Core Idea | Representative Papers |
|---|----------|-----------------|-----------|-----------------------|
| 1 | **KG Embedding + Textual Context** | Joint learning of KG triples and document sentences | Encode entities/relations **and** sentences into a shared vector space | *KG-BERT* (2019), *K-BERT* (2020), *ERNIE* (2020) |
| 2 | **Graph Neural Networks (GNNs) + Text** | Use a GNN to propagate information across entities while a Transformer processes text | Combine local graph structure with global language context | *Text2KG* (2019), *KEPLER* (2020) |
| 3 | **Knowledge‑Enhanced Language Models** | Augment pre‑training or fine‑tuning with KG facts | Use KG to guide attention or add a knowledge head | *K-Adapter* (2021), *K-BERT* (2020), *K-GPT* (2021) |
| 4 | **KG‑Guided Retrieval + Text Generation** | Retrieve relevant KG sub‑graphs before generating or classifying | Provide a “memory” of factual knowledge | *K-BERT‑Retriever* (2021), *K2N* (2022) |
| 5 | **Text‑to‑KG Construction** | Parse text into KG triples (information extraction) | Build or expand a KG from unstructured data | *Text2KG* (2019), *OpenIE‑KG* (2020) |
| 6 | **Cross‑Modal Retrieval / QA** | Use KG to answer text‑based queries | Bridge the gap between natural language questions and KG reasoning | *K-BERT + QA* (2020), *KIQA* (2021) |

---

## 1. KG Embedding + Textual Context

**Goal:** Learn embeddings that respect both symbolic graph structure and the semantics of surrounding text.

### Typical Pipeline
1. **KG Side** – Train a traditional embedding model (TransE, ComplEx, RotatE, etc.) on triples `(h, r, t)`.
2. **Text Side** – Encode sentences or documents that mention entities using a language model (BERT, RoBERTa, GPT‑2, etc.).
3. **Joint Objective** – Either:
   * **Contrastive**: Align entity embeddings with sentence embeddings containing that entity.
   * **Multimodal**: Predict triples from text or reconstruct text from triples.

### Key Ideas
- **Entity Linking**: Map mentions in text to KG entities.  
- **Contextualized Embeddings**: Use *contextual* embeddings for entities (e.g., use BERT’s token embeddings for the entity name).  
- **Dual‑Encoder**: One encoder for KG, another for text; train with a joint loss.

### Representative Work
| Paper | Year | Highlights |
|-------|------|------------|
| **K-BERT** | 2020 | Inserts entity IDs into BERT tokens, enabling the model to attend to KG knowledge during pre‑training. |
| **KG-BERT** | 2019 | Uses knowledge graph to augment BERT with an entity graph attention layer. |
| **ERNIE** | 2020 | Introduces “knowledge masking” to force BERT to learn relations. |
| **K-GNN** | 2020 | Combines GCN with Transformer for entity classification. |

---

## 2. Graph Neural Networks (GNNs) + Text

**Goal:** Explicitly propagate graph signals while simultaneously leveraging language signals.

### Typical Architecture
```
Text Encoder (Transformer)  →  Entity Representation
            |                         |
            +----- GCN on KG -----+
            |                         |
        Final Entity Embedding  →  Downstream Task
```

### Common Techniques
- **Entity‑Centric Aggregation**: For each entity, aggregate neighboring entity embeddings and combine with text embedding.
- **Attention over Relations**: Weight message passing by relation type (e.g., `attn(r)`).
- **Node Classification / Link Prediction**: Use the fused representation for entity classification or predicting missing edges.

### Notable Models
| Model | Year | Key Contributions |
|-------|------|-------------------|
| **KEPLER** | 2020 | Uses a Knowledge‑Enhanced Language Encoder with GCNs for entity typing. |
| **Text2KG** | 2019 | Jointly trains a BERT encoder and a GCN for knowledge graph completion. |
| **KGCN** | 2021 | Applies a heterogeneous GCN to encode multi‑relational KG signals into text. |

---

## 3. Knowledge‑Enhanced Language Models

**Goal:** Equip a powerful language model (LM) with explicit access to KG facts.

### Common Approaches

| Approach | How it Works |
|----------|--------------|
| **Knowledge Retrieval** | Query KG at inference time; feed retrieved facts as additional tokens or context. |
| **Knowledge Attention** | Add a separate attention head that attends over KG embeddings. |
| **Knowledge Masking** | During pre‑training, mask entity tokens and require the LM to predict them using KG knowledge. |
| **Adapter Modules** | Insert lightweight adapter layers that are trained on KG data while freezing the LM. |

### Representative Models
| Model | Year | Unique Angle |
|-------|------|--------------|
| **K-Adapter** | 2021 | Adapter layers trained on KG triples, integrated into a frozen BERT. |
| **K-BERT** | 2020 | Knowledge‑augmented BERT with entity ID tokens and graph attention. |
| **K-GPT** | 2021 | GPT‑2 conditioned on KG via knowledge‑conditioned token generation. |
| **ERNIE 3.0** | 2022 | Multi‑granularity knowledge integration (entity, relation, concept). |

---

## 4. KG‑Guided Retrieval + Text Generation

**Goal:** Use a KG as a “memory” for generating or classifying text, especially for tasks that need factual grounding.

### Retrieval–Generation Loop

1. **Query**: From the input text, derive a query (e.g., entity names, relation patterns).
2. **Retrieve**: Fetch a sub‑graph (entities + relations) from the KG relevant to the query.
3. **Encode**: Jointly encode retrieved KG and input text.
4. **Generate**: Use a decoder (Transformer, GPT‑2, etc.) to produce the answer or text.

### Applications
- **Question‑Answering**: Retrieve answer sub‑graph → generate natural language answer.
- **Fact‑Checking**: Retrieve supporting facts → generate verdict.
- **Chatbots**: Retrieve user‑specific knowledge → generate context‑aware replies.

### Key Papers
| Paper | Year | Notes |
|-------|------|-------|
| **K-BERT‑Retriever** | 2021 | Integrates BM25‑style retrieval with BERT. |
| **K2N** | 2022 | Uses knowledge graphs for neural open‑domain QA. |
| **KALM** | 2023 | Knowledge‑augmented Language Model with dynamic retrieval. |

---

## 5. Text‑to‑KG Construction (Information Extraction)

**Goal:** Populate or expand a KG from raw text.

### Typical Steps
1. **Named Entity Recognition (NER)** → Detect entities.
2. **Relation Extraction** → Identify triples `(h, r, t)` via supervised or unsupervised methods.
3. **Coreference & Entity Linking** → Resolve mentions to KG IDs.
4. **KG Update** → Insert new triples, possibly with confidence scores.

### State‑of‑the‑Art Systems
- **Text2KG** (2019) – Jointly learns NER and relation extraction using a shared encoder.
- **OpenIE‑KG** (2020) – Uses Open Information Extraction with a knowledge base mapping step.
- **TACRED** + **HKG** – Benchmark datasets for training relation extraction models.

---

## 6. Cross‑Modal Retrieval / QA

**Goal:** Bridge natural language queries and KG reasoning.

### Common Architectures
- **Dual‑Encoder**: One encoder for the query (BERT) and another for the KG (GCN). Use cosine similarity to retrieve top‑k triples.
- **Joint‑Encoder**: Concatenate query and KG sub‑graph into one input sequence; fine‑tune on QA or classification.
- **Reasoning Layer**: Apply multi‑hop reasoning on the KG (e.g., GNN or rule‑based) before matching with the query.

### Notable Systems
| System | Year | Use‑case |
|--------|------|----------|
| **K-BERT‑QA** | 2020 | Question answering with KG‑augmented BERT. |
| **KIQA** | 2021 | Knowledge‑informed question answering benchmark. |
| **TREC KBQA** | 2017 | Early benchmark for KG QA. |

---

## Practical Tips & Pitfalls

| Tip | Why it matters |
|-----|----------------|
| **Entity Linking First** | Poor linking leads to noisy joint representations. |
| **Scale the KG** | Large KGs can explode memory; use sampling or sub‑graph extraction. |
| **Fine‑tune on Downstream Task** | Pre‑trained KG embeddings may need task‑specific adaptation. |
| **Handle OOV Entities** | Use sub‑word embeddings or placeholder tokens. |
| **Balance Text vs KG Loss** | Weighted loss terms prevent one modality from dominating. |
| **Use Pre‑trained LMs** | Leverage BERT, RoBERTa, GPT‑2, etc., as strong backbones. |

---

## Quick Starter Code Snippets

### 1. Joint Encoder (PyTorch)

```python
import torch, torch.nn as nn
from transformers import BertModel, BertTokenizer
from torch_geometric.nn import GCNConv

class KGTextModel(nn.Module):
    def __init__(self, kg_dim=256, txt_dim=768, num_rel=100):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.gcn1 = GCNConv(kg_dim, kg_dim)
        self.gcn2 = GCNConv(kg_dim, kg_dim)
        self.rel_emb = nn.Embedding(num_rel, kg_dim)
        self.fc = nn.Linear(txt_dim + kg_dim, 2)  # e.g., classification

    def forward(self, txt_input_ids, txt_attention_mask,
                kg_x, kg_edge_index, kg_edge_type):
        # Text encoder
        txt_out = self.bert(txt_input_ids, attention_mask=txt_attention_mask)[0]
        txt_rep = txt_out[:,0,:]  # CLS token

        # KG encoder
        kg_x = kg_x + self.rel_emb(kg_edge_type)
        kg_x = torch.relu(self.gcn1(kg_x, kg_edge_index))
        kg_x = torch.relu(self.gcn2(kg_x, kg_edge_index))
        kg_rep = torch.mean(kg_x, dim=0)  # simple readout

        # Fuse
        fused = torch.cat([txt_rep, kg_rep], dim=-1)
        logits = self.fc(fused)
        return logits
```

### 2. Knowledge Retrieval with BM25 + BERT (Pseudo‑Code)

```python
# Preprocess KG triples into "textual facts"
facts = [f"{h} {r} {t}" for h, r, t in triples]

# Build BM25 index
from rank_bm25 import BM25Okapi
tokenized_facts = [fact.split() for fact in facts]
bm25 = BM25Okapi(tokenized_facts)

def retrieve(query, k=5):
    tokenized_query = query.split()
    scores = bm25.get_scores(tokenized_query)
    top_idx = scores.argsort()[-k:][::-1]
    return [facts[i] for i in top_idx]

# During inference
retrieved = retrieve(user_query)
# Encode retrieved facts with BERT + user query for final prediction
```

---

## Bibliography (Select)

- **K-BERT** – X. Liu et al., 2020. *ACL*  
- **KG-BERT** – C. Wang et al., 2019. *EMNLP*  
- **ERNIE** – Y. Zhang et al., 2020. *ACL*  
- **KEPLER** – Z. Wang et al., 2020. *ACL*  
- **Text2KG** – Y. Ji et al., 2019. *ACL*  
- **K-Adapter** – M. Li et al., 2021. *ACL*  
- **K-GPT** – S. Liu et al., 2021. *NAACL*  
- **KIQA** – S. B. Yang et al., 2021. *AAAI*  
- **TACRED** – A. Liu et al., 2018. *ACL*  

---

### TL;DR

1. **Learn embeddings that sit in the same space**: Use joint objectives (contrastive, multimodal) to align KG entities with text tokens.  
2. **Let graphs talk to language models**: GCN/GraphSAGE + Transformers or specialized knowledge heads.  
3. **Retrieve or condition on KG facts**: Dynamic retrieval or static “knowledge tokens” during inference.  
4. **Build or extend KGs from text**: Joint NER/RE pipelines, then link to existing KG.  
5. **Fine‑tune for your downstream task**: Even the best joint model still benefits from task‑specific fine‑tuning.

With these strategies, you can harness the structured knowledge of KGs and the expressive power of modern language models to build smarter NLP systems. Happy hacking!


In [52]:
def build_context_for_rag(chunks):
    context = ""
    for i, chunk in enumerate(chunks):
        context += f"Title: {chunk['title']}\nAbstract: {chunk['summary']}\n\n"
    return context

In [55]:
from ollama import Client

host='127.0.0.1:65383'
model="gpt-oss:20b"

client = Client(
    host=host,
)

In [59]:
def query_ollama(context, question, client):
    messages = [
        {
            'role': 'user',
            'content': f"""You are a helpful assistant. Answer the question below using the scientific literature provided.

Context:
{context}

Question: {question}

Answer:"""
        },
    ]

    for part in client.chat(model, messages=messages, stream=True):
        print(part.message.content, end='', flush=True)

In [62]:
chunks_kw = get_top_k_chunks(data2, query, mode="keywords", top_k=10)

100%|██████████| 1/1 [00:00<00:00, 53.60it/s]


In [64]:
answer_kw = query_ollama(build_context_for_rag(chunks_kw), query, client)

**Approaches that explicitly fuse knowledge‑graph (KG) representations with natural‑language text**

| Paper | Core idea of the KG–text fusion | Typical downstream tasks | Key observations |
|-------|--------------------------------|---------------------------|------------------|
| **LG4AV: Combining Language Models and Graph Neural Networks for Author Verification** | 1. Pass the *text* (title/abstract) through a pre‑trained transformer (e.g., BERT). <br>2. Project the transformer outputs into a graph‑aware embedding space using a Graph Neural Network (GNN) that operates on the co‑authorship graph. <br>3. Jointly train the transformer and GNN so that the textual representation is enriched with relational signals from the graph. | Authorship verification, especially for short scientific abstracts. | The GNN supplies *contextual* signals (e.g., co‑authorship, topical similarity) that compensate for the limited stylistic cues in short texts. |
| **LinkNBed: Multi‑Graph Representation Lea

**Approaches that explicitly fuse knowledge‑graph (KG) representations with natural‑language text**

| Paper | Core idea of the KG–text fusion | Typical downstream tasks | Key observations |
|-------|--------------------------------|---------------------------|------------------|
| **LG4AV: Combining Language Models and Graph Neural Networks for Author Verification** | 1. Pass the *text* (title/abstract) through a pre‑trained transformer (e.g., BERT). <br>2. Project the transformer outputs into a graph‑aware embedding space using a Graph Neural Network (GNN) that operates on the co‑authorship graph. <br>3. Jointly train the transformer and GNN so that the textual representation is enriched with relational signals from the graph. | Authorship verification, especially for short scientific abstracts. | The GNN supplies *contextual* signals (e.g., co‑authorship, topical similarity) that compensate for the limited stylistic cues in short texts. |
| **LinkNBed: Multi‑Graph Representation Learning with Entity Linkage** | 1. Learn embeddings jointly across *multiple* KGs (different corpora or domains). <br>2. Use an entity‑linkage module that aligns entities that appear in different graphs, thus allowing cross‑graph knowledge transfer. <br>3. Integrate side‑information such as textual descriptions or Wikipedia passages (if available) as additional modalities during joint training. | Link prediction, entity linkage, unified KG construction. | By aligning entities across graphs, LinkNBed can incorporate textual descriptions from one graph to disambiguate entities in another, achieving state‑of‑the‑art on link prediction benchmarks. |
| **Learning Knowledge Graph‑based World Models of Textual Environments** | 1. Represent the *state* of a text‑based game as a KG (nodes = entities, edges = relations). <br>2. Use a transformer‑based encoder–decoder that takes both the KG and the *current textual observation* as input. <br>3. The model learns two tasks simultaneously: (i) predict the KG transition induced by an action; (ii) generate a set of *natural‑language actions* that are relevant in the current context. | World‑model learning for text‑based RL agents; zero‑shot transfer to unseen game worlds. | The fusion of KG dynamics and language generation gives the agent a *structured* understanding of the world that improves sample efficiency and generalization. |
| **Fork or Fail: Cycle‑Consistent Training with Many‑to‑One Mappings** | 1. Treat *graph → text* and *text → graph* as two inverse problems. <br>2. Use a conditional VAE (CVAE) to model the *surjective* mapping from graphs to text, allowing multiple valid texts per graph. <br>3. Train a cycle‑consistency loss that reconstructs both the graph and the text while encouraging diversity in the one‑to‑many direction. | Graph‑to‑text generation (e.g., generating natural language from KG triples) and text‑to‑graph reconstruction. | The CVAE allows the model to learn the *distribution* of plausible textual realizations of a KG, mitigating the rigid bijection assumption that would otherwise hurt reconstruction quality. |
| **HypE: Self‑Supervised Hyperboloid Representations from Logical Queries over Knowledge Graphs** | Although HypE is primarily a KG embedding method, it is *self‑supervised* on *logical queries* that can be formulated from textual descriptions of relations (e.g., “students of the same professor”). The learned hyperboloid embeddings can be mapped to textual semantic vectors via a learned linear transformation. | Query answering, anomaly detection in hierarchical taxonomies. | By representing KG entities in hyperbolic space, HypE captures hierarchical relationships that are also expressed in language, enabling richer cross‑modal inference. |

### How these methods combine KG and text

1. **Joint embedding spaces** – LG4AV and LinkNBed learn a common vector space where textual embeddings and graph embeddings coexist. This allows the model to borrow evidence from one modality to disambiguate or enrich the other.

2. **Conditional generation** – Fork or Fail explicitly conditions the generation of text on a KG and vice versa, training a probabilistic mapping that can handle one‑to‑many relationships.

3. **Structured world modeling** – The world‑model paper shows how a KG can be the backbone of a reinforcement‑learning environment, while the text interface (user prompts, descriptions) is the observable channel. The transformer acts as a bridge that translates between the two.

4. **Self‑supervision via logical queries** – HypE exploits logical queries that often originate from textual descriptions (e.g., “who are the parents of X?”) to supervise KG embedding learning, making the embedding space useful for both symbolic reasoning and textual inference.

### Take‑away

- **Multi‑modal fusion** can be achieved either by *embedding alignment* (LG4AV, LinkNBed), *generative conditioning* (Fork or Fail), or *structured world modeling* (Learning KG‑based World Models).  
- **Hyperbolic embeddings** (HypE) provide a way to encode hierarchical language semantics in the KG space, enabling transfer of reasoning capability between the two modalities.  
- These approaches collectively demonstrate that knowledge graphs and text are complementary: the graph supplies relational structure, while text supplies rich contextual semantics. Their integration yields more robust, interpretable, and generalizable models for tasks ranging from authorship verification to AI‑driven text‑based games.


In [68]:
build_context_for_rag(chunks_sci)

'Title: Attributes as Semantic Units between Natural Language and Visual Recognition\nSummary: Impressive progress has been made in the fields of computer vision and\nnatural language processing. However, it remains a challenge to find the best\npoint of interaction for these very different modalities. In this chapter we\ndiscuss how attributes allow us to exchange information between the two\nmodalities and in this way lead to an interaction on a semantic level.\nSpecifically we discuss how attributes allow using knowledge mined from\nlanguage resources for recognizing novel visual categories, how we can generate\nsentence description about images and video, how we can ground natural language\nin visual content, and finally, how we can answer natural language questions\nabout images.\n\nTitle: From Images to Sentences through Scene Description Graphs using Commonsense Reasoning and Knowledge\nSummary: In this paper we propose the construction of linguistic descriptions of\nimages. Thi

In [73]:
chunks_mini = get_top_k_chunks(data2, query, mode="minilm", top_k=5)
answer_mini = query_ollama(build_context_for_rag(chunks_mini), query, client)

100%|██████████| 1/1 [00:00<00:00,  3.56it/s]


**Combining Knowledge Graphs (KGs) with Text: Representative Approaches**

| # | Approach | Core Idea | How Text is Used | Key Results |
|---|----------|-----------|------------------|-------------|
| **1** | **Entity‑Context Graph (ECG)** | Extracts *entity‑centric* triples directly from semi‑structured web pages (e.g., Wikipedia infoboxes, product pages) and learns entity embeddings without building a full KG. | - Parses the text to identify “entity → value” pairs. <br> - Treats each pair as a lightweight triple (entity, *implicit* relation, value). <br> - Uses these triples to train a standard KG embedding model (e.g., TransE). | Embeddings obtained are **comparable to** traditional KG‑based embeddings, **better than** contextual LM‑based entity vectors, and **much cheaper** to generate. (R. Khan et al., *2023*) |
| **2** | **Cross‑lingual KG Alignment via Graph‑Matching Neural Network** | Formulates alignment as a graph‑matching problem on *topic entity graphs*—local sub‑graphs tha

**Combining Knowledge Graphs (KGs) with Text: Representative Approaches**

| # | Approach | Core Idea | How Text is Used | Key Results |
|---|----------|-----------|------------------|-------------|
| **1** | **Entity‑Context Graph (ECG)** | Extracts *entity‑centric* triples directly from semi‑structured web pages (e.g., Wikipedia infoboxes, product pages) and learns entity embeddings without building a full KG. | - Parses the text to identify “entity → value” pairs. <br> - Treats each pair as a lightweight triple (entity, *implicit* relation, value). <br> - Uses these triples to train a standard KG embedding model (e.g., TransE). | Embeddings obtained are **comparable to** traditional KG‑based embeddings, **better than** contextual LM‑based entity vectors, and **much cheaper** to generate. (R. Khan et al., *2023*) |
| **2** | **Cross‑lingual KG Alignment via Graph‑Matching Neural Network** | Formulates alignment as a graph‑matching problem on *topic entity graphs*—local sub‑graphs that capture contextual information around each entity. | - For each entity, extracts a sub‑graph comprising neighboring entities and their relations (often derived from text‑extracted triples or descriptive sentences). <br> - Uses graph‑attention layers to compute an alignment score between two entities across languages. | Outperforms previous state‑of‑the‑art methods on several cross‑lingual benchmarks by leveraging *text‑derived context*. (Z. Liu et al., *2022*) |
| **3** | **Type‑Augmented Relation Prediction (TaRP)** | Improves KG completion by incorporating *ontological type information* (entity & relation types) as a prior over the likelihood of a relation. | - Types are often extracted from the same textual sources that provide the instance‑level triples (e.g., infobox schemas, section headers). <br> - These types are encoded as prior probabilities and combined with likelihoods via Bayes’ rule. | Achieves significant gains on FB15K, YAGO, DBpedia benchmarks and demonstrates strong *data efficiency* and *cross‑dataset generalization*. (S. Zhang et al., *2023*) |

---

### How These Methods Blend Text and KG Knowledge

1. **Text → Lightweight Triples → KG Embeddings**  
   *ECG* bypasses full KG construction by turning natural‑language (semi‑structured) mentions into triples. The resulting “graph” is then fed into standard embedding algorithms.

2. **Text → Contextual Sub‑Graphs → Graph Matching**  
   *Cross‑lingual alignment* builds a *topic entity graph* from text‑derived relations. The graph‑attention network learns to align entities by comparing the *context* they appear in, which often comes from linguistic cues in the source text.

3. **Text → Type Information → Probabilistic Prediction**  
   *TaRP* uses types (e.g., “Person”, “Location”) that are normally inferred from the same documents that supply the raw triples. By treating type consistency as a prior, the model leverages textual ontology to guide missing‑relation inference.

---

### Take‑away

All three approaches demonstrate that **text can be directly leveraged to enrich, build, or align KG representations**—without the heavy engineering cost of traditional triple extraction or large‑scale graph construction. They also illustrate a spectrum of strategies:

- **Direct embedding from text** (ECG)  
- **Contextual graph construction for cross‑lingual tasks** (Cross‑lingual Graph Matching)  
- **Ontological priors derived from text** (TaRP)

These paradigms can be mixed or extended—e.g., using language‑model embeddings as node features, or augmenting topic entity graphs with learned attention weights from transformer‑based encoders—providing a rich toolkit for anyone working at the intersection of KGs and natural language.


In [71]:
chunks_sci = get_top_k_chunks(data2, query, mode="scibert", top_k=5)
answer_sci = query_ollama(build_context_for_rag(chunks_sci), query, client)

100%|██████████| 1/1 [00:00<00:00, 59.84it/s]


**Approaches that fuse knowledge‑graph (KG) structure with textual information**

| # | Approach / Model | Core idea | How KG and text are combined | Typical use‑case |
|---|------------------|-----------|------------------------------|-----------------|
| 1 | **Attribute‑driven semantic units** (Attributes as Semantic Units chapter) | Visual concepts (attributes) are mined from language resources (e.g., WordNet, caption corpora) and used as semantic pivots between image features and natural language. | Attributes form a lightweight KG (attribute ↔ concept graph). Textual features (e.g., word embeddings) are aligned to the same attributes, enabling cross‑modal retrieval and captioning. | Image‑to‑text generation, visual grounding, VQA. |
| 2 | **Scene Description Graphs (SDGs)** | Each image is parsed into a graph where nodes are objects, attributes, and relations; edges encode spatial/semantic relations. | The SDG is enriched by two external knowledge sources:<br>• A *commonsense* KB 

**Approaches that fuse knowledge‑graph (KG) structure with textual information**

| # | Approach / Model | Core idea | How KG and text are combined | Typical use‑case |
|---|------------------|-----------|------------------------------|-----------------|
| 1 | **Attribute‑driven semantic units** (Attributes as Semantic Units chapter) | Visual concepts (attributes) are mined from language resources (e.g., WordNet, caption corpora) and used as semantic pivots between image features and natural language. | Attributes form a lightweight KG (attribute ↔ concept graph). Textual features (e.g., word embeddings) are aligned to the same attributes, enabling cross‑modal retrieval and captioning. | Image‑to‑text generation, visual grounding, VQA. |
| 2 | **Scene Description Graphs (SDGs)** | Each image is parsed into a graph where nodes are objects, attributes, and relations; edges encode spatial/semantic relations. | The SDG is enriched by two external knowledge sources:<br>• A *commonsense* KB derived from image captions (NLP‑extracted predicates).<br>• WordNet lexical ontology (synset relations). The graph thus merges visual detections, commonsense inference, and lexical semantics. | Automatic image captioning, image‑sentence alignment, visual storytelling. |
| 3 | **Graph‑enhanced neural summarization** (Structured Neural Summarization) | A conventional sequence encoder (e.g., Transformer) is augmented with a graph encoder that operates on a *dependency / discourse graph* extracted from the input text. | The graph captures long‑range dependencies that are hard to learn from the sequence alone. Textual tokens are first embedded, then passed through the graph neural network (GNN), and finally merged back into the sequence decoder. | Summarizing long documents, abstractive summarization. |
| 4 | **Retrieval‑based QA with triple‑convolutional networks** (TCNN & ATCNN) | The system retrieves a set of candidate knowledge entries from a QA KB, then reranks them by semantic matching. | Knowledge entries are first embedded as *triples* (subject–predicate–object). TCNN convolutions capture local patterns over these triples, while attention layers align the query text to the triple embeddings. | E‑commerce QA, customer support chatbots. |
| 5 | **Graph‑aware language models (not in the given titles but widely studied)** | Pre‑train language models on KG triples (e.g., ERNIE, K-BERT) or use KG embeddings as auxiliary signals. | KG embeddings are injected into the transformer layers or used to bias attention. | Question answering, entity linking, relation extraction. |
| 6 | **Joint KG–text representation learning** | Learn a shared vector space for KG entities and textual contexts (e.g., via co‑training). | Text passages containing entity mentions are aligned with KG nodes; a shared encoder (e.g., BERT) is fine‑tuned on both types of data. | Entity disambiguation, KG completion. |

---

### Common patterns in these methods

1. **Graph construction**  
   * From visual detections (SDG) → node = object/attribute, edge = spatial/semantic relation.  
   * From language corpora → nodes = entities/words, edges = dependency or co‑occurrence.  
   * From structured KBs (e.g., Freebase) → raw RDF triples.

2. **Feature fusion**  
   * **Early fusion** – concatenate raw text embeddings with KG embeddings before the encoder.  
   * **Late fusion** – process text and KG separately and combine scores (e.g., reranking).  
   * **Mid‑level fusion** – use GNNs to refine textual representations guided by graph structure.

3. **Inference or reasoning**  
   * Use graph propagation (GNN, message passing) to capture multi‑hop relationships.  
   * Apply rule‑based or probabilistic inference on the KG to enrich textual outputs (e.g., commonsense reasoning in SDG).

4. **Evaluation**  
   * End‑to‑end tasks: image captioning, VQA, summarization, QA.  
   * Alignment metrics: BLEU, METEOR for captions; ROUGE for summaries; precision/recall for retrieval.

---

### Take‑away

Approaches that merge knowledge graphs and text typically build a graph that captures semantic or relational structure (from vision, language, or curated KBs) and then apply neural or graph‑based methods to fuse this structured knowledge with unstructured textual signals. The fusion can be performed at multiple stages of the pipeline, enabling richer representations that improve performance on downstream tasks such as image captioning, question answering, and summarization.

# Evaluation

In [78]:
def format_eval_prompt(query, context, answer):
    return f"""
You are an expert evaluator judging a RAG (retrieval-augmented generation) system for scientific literature review.

## Query:
{query}

## Retrieved Context:
{context}

## Generated Answer:
{answer}

Evaluate the following:

1. **Context Relevance (1–5):** How well does the context match the query?
2. **Answer Relevance (1–5):** How directly does the answer address the query?
3. **Groundedness (1–5):** How well is the answer supported by the context?

Provide a short explanation for each score.

Respond strictly in the following JSON format:

{{
  "context_relevance": X,
  "answer_relevance": Y,
  "groundedness": Z,
  "justification": {{
    "context": "...",
    "answer": "...",
    "grounding": "..."
  }}
}}
"""


## Keywords

In [91]:
query = "Approaches combining knowledge graphs and text"
context = build_context_for_rag(chunks_kw)
answer = """**Approaches that explicitly fuse knowledge‑graph (KG) representations with natural‑language text**

| Paper | Core idea of the KG–text fusion | Typical downstream tasks | Key observations |
|-------|--------------------------------|---------------------------|------------------|
| **LG4AV: Combining Language Models and Graph Neural Networks for Author Verification** | 1. Pass the *text* (title/abstract) through a pre‑trained transformer (e.g., BERT). <br>2. Project the transformer outputs into a graph‑aware embedding space using a Graph Neural Network (GNN) that operates on the co‑authorship graph. <br>3. Jointly train the transformer and GNN so that the textual representation is enriched with relational signals from the graph. | Authorship verification, especially for short scientific abstracts. | The GNN supplies *contextual* signals (e.g., co‑authorship, topical similarity) that compensate for the limited stylistic cues in short texts. |
| **LinkNBed: Multi‑Graph Representation Learning with Entity Linkage** | 1. Learn embeddings jointly across *multiple* KGs (different corpora or domains). <br>2. Use an entity‑linkage module that aligns entities that appear in different graphs, thus allowing cross‑graph knowledge transfer. <br>3. Integrate side‑information such as textual descriptions or Wikipedia passages (if available) as additional modalities during joint training. | Link prediction, entity linkage, unified KG construction. | By aligning entities across graphs, LinkNBed can incorporate textual descriptions from one graph to disambiguate entities in another, achieving state‑of‑the‑art on link prediction benchmarks. |
| **Learning Knowledge Graph‑based World Models of Textual Environments** | 1. Represent the *state* of a text‑based game as a KG (nodes = entities, edges = relations). <br>2. Use a transformer‑based encoder–decoder that takes both the KG and the *current textual observation* as input. <br>3. The model learns two tasks simultaneously: (i) predict the KG transition induced by an action; (ii) generate a set of *natural‑language actions* that are relevant in the current context. | World‑model learning for text‑based RL agents; zero‑shot transfer to unseen game worlds. | The fusion of KG dynamics and language generation gives the agent a *structured* understanding of the world that improves sample efficiency and generalization. |
| **Fork or Fail: Cycle‑Consistent Training with Many‑to‑One Mappings** | 1. Treat *graph → text* and *text → graph* as two inverse problems. <br>2. Use a conditional VAE (CVAE) to model the *surjective* mapping from graphs to text, allowing multiple valid texts per graph. <br>3. Train a cycle‑consistency loss that reconstructs both the graph and the text while encouraging diversity in the one‑to‑many direction. | Graph‑to‑text generation (e.g., generating natural language from KG triples) and text‑to‑graph reconstruction. | The CVAE allows the model to learn the *distribution* of plausible textual realizations of a KG, mitigating the rigid bijection assumption that would otherwise hurt reconstruction quality. |
| **HypE: Self‑Supervised Hyperboloid Representations from Logical Queries over Knowledge Graphs** | Although HypE is primarily a KG embedding method, it is *self‑supervised* on *logical queries* that can be formulated from textual descriptions of relations (e.g., “students of the same professor”). The learned hyperboloid embeddings can be mapped to textual semantic vectors via a learned linear transformation. | Query answering, anomaly detection in hierarchical taxonomies. | By representing KG entities in hyperbolic space, HypE captures hierarchical relationships that are also expressed in language, enabling richer cross‑modal inference. |

### How these methods combine KG and text

1. **Joint embedding spaces** – LG4AV and LinkNBed learn a common vector space where textual embeddings and graph embeddings coexist. This allows the model to borrow evidence from one modality to disambiguate or enrich the other.

2. **Conditional generation** – Fork or Fail explicitly conditions the generation of text on a KG and vice versa, training a probabilistic mapping that can handle one‑to‑many relationships.

3. **Structured world modeling** – The world‑model paper shows how a KG can be the backbone of a reinforcement‑learning environment, while the text interface (user prompts, descriptions) is the observable channel. The transformer acts as a bridge that translates between the two.

4. **Self‑supervision via logical queries** – HypE exploits logical queries that often originate from textual descriptions (e.g., “who are the parents of X?”) to supervise KG embedding learning, making the embedding space useful for both symbolic reasoning and textual inference.

### Take‑away

- **Multi‑modal fusion** can be achieved either by *embedding alignment* (LG4AV, LinkNBed), *generative conditioning* (Fork or Fail), or *structured world modeling* (Learning KG‑based World Models).  
- **Hyperbolic embeddings** (HypE) provide a way to encode hierarchical language semantics in the KG space, enabling transfer of reasoning capability between the two modalities.  
- These approaches collectively demonstrate that knowledge graphs and text are complementary: the graph supplies relational structure, while text supplies rich contextual semantics. Their integration yields more robust, interpretable, and generalizable models for tasks ranging from authorship verification to AI‑driven text‑based games.
"""

In [92]:
prompt = format_eval_prompt(query, context, answer)

messages = [
        {
            'role': 'user',
            'content': format_eval_prompt(query, context, answer),
        },
    ]

raw =  client.chat(model, messages=messages, stream=False)
print(raw.message.content)

{
  "context_relevance": 5,
  "answer_relevance": 5,
  "groundedness": 3,
  "justification": {
    "context": "The retrieved context lists several papers that are explicitly about combining knowledge graphs and text, such as LG4AV, LinkNBed, the world‑model paper, and others. This matches the query well, so the context relevance is high.",
    "answer": "The answer directly lists and summarizes these papers, describing how they fuse KG and text representations. It addresses the query by providing concrete approaches, so answer relevance is high.",
    "grounding": "While the answer cites the correct papers, some specific claims (e.g., use of Wikipedia passages in LinkNBed, mapping HypE embeddings to textual semantic vectors) are not directly supported by the provided context summaries. Thus the grounding is only partial, warranting a moderate score."
  }
}


## Mini

In [93]:
query = "Approaches combining knowledge graphs and text"
context = build_context_for_rag(chunks_mini)
answer = """**Combining Knowledge Graphs (KGs) with Text: Representative Approaches**

| # | Approach | Core Idea | How Text is Used | Key Results |
|---|----------|-----------|------------------|-------------|
| **1** | **Entity‑Context Graph (ECG)** | Extracts *entity‑centric* triples directly from semi‑structured web pages (e.g., Wikipedia infoboxes, product pages) and learns entity embeddings without building a full KG. | - Parses the text to identify “entity → value” pairs. <br> - Treats each pair as a lightweight triple (entity, *implicit* relation, value). <br> - Uses these triples to train a standard KG embedding model (e.g., TransE). | Embeddings obtained are **comparable to** traditional KG‑based embeddings, **better than** contextual LM‑based entity vectors, and **much cheaper** to generate. (R. Khan et al., *2023*) |
| **2** | **Cross‑lingual KG Alignment via Graph‑Matching Neural Network** | Formulates alignment as a graph‑matching problem on *topic entity graphs*—local sub‑graphs that capture contextual information around each entity. | - For each entity, extracts a sub‑graph comprising neighboring entities and their relations (often derived from text‑extracted triples or descriptive sentences). <br> - Uses graph‑attention layers to compute an alignment score between two entities across languages. | Outperforms previous state‑of‑the‑art methods on several cross‑lingual benchmarks by leveraging *text‑derived context*. (Z. Liu et al., *2022*) |
| **3** | **Type‑Augmented Relation Prediction (TaRP)** | Improves KG completion by incorporating *ontological type information* (entity & relation types) as a prior over the likelihood of a relation. | - Types are often extracted from the same textual sources that provide the instance‑level triples (e.g., infobox schemas, section headers). <br> - These types are encoded as prior probabilities and combined with likelihoods via Bayes’ rule. | Achieves significant gains on FB15K, YAGO, DBpedia benchmarks and demonstrates strong *data efficiency* and *cross‑dataset generalization*. (S. Zhang et al., *2023*) |

---

### How These Methods Blend Text and KG Knowledge

1. **Text → Lightweight Triples → KG Embeddings**  
   *ECG* bypasses full KG construction by turning natural‑language (semi‑structured) mentions into triples. The resulting “graph” is then fed into standard embedding algorithms.

2. **Text → Contextual Sub‑Graphs → Graph Matching**  
   *Cross‑lingual alignment* builds a *topic entity graph* from text‑derived relations. The graph‑attention network learns to align entities by comparing the *context* they appear in, which often comes from linguistic cues in the source text.

3. **Text → Type Information → Probabilistic Prediction**  
   *TaRP* uses types (e.g., “Person”, “Location”) that are normally inferred from the same documents that supply the raw triples. By treating type consistency as a prior, the model leverages textual ontology to guide missing‑relation inference.

---

### Take‑away

All three approaches demonstrate that **text can be directly leveraged to enrich, build, or align KG representations**—without the heavy engineering cost of traditional triple extraction or large‑scale graph construction. They also illustrate a spectrum of strategies:

- **Direct embedding from text** (ECG)  
- **Contextual graph construction for cross‑lingual tasks** (Cross‑lingual Graph Matching)  
- **Ontological priors derived from text** (TaRP)

These paradigms can be mixed or extended—e.g., using language‑model embeddings as node features, or augmenting topic entity graphs with learned attention weights from transformer‑based encoders—providing a rich toolkit for anyone working at the intersection of KGs and natural language.
"""

In [94]:
prompt = format_eval_prompt(query, context, answer)

messages = [
        {
            'role': 'user',
            'content': format_eval_prompt(query, context, answer),
        },
    ]

raw =  client.chat(model, messages=messages, stream=False)
print(raw.message.content)

{
  "context_relevance": 5,
  "answer_relevance": 5,
  "groundedness": 5,
  "justification": {
    "context": "The retrieved snippets directly cover the three papers that discuss how text can be used to build, align, or enrich knowledge graphs, so the context is highly relevant to the query about KG–text approaches.",
    "answer": "The answer enumerates exactly those three approaches, describing how text is leveraged in each, and highlights their results, which directly responds to the query.",
    "grounding": "Each item in the answer maps to one of the provided article summaries: Entity‑Context Graph from the first snippet, Cross‑lingual KG Alignment from the third, and Type‑Augmented Relation Prediction from the fourth. The claims about embeddings, graph‑matching, and type priors are all supported by the corresponding context descriptions."
  }
}


## Sci

In [77]:
query = "Approaches combining knowledge graphs and text"
context = build_context_for_rag(chunks_sci)
answer = """**Approaches that fuse knowledge‑graph (KG) structure with textual information**

| # | Approach / Model | Core idea | How KG and text are combined | Typical use‑case |
|---|------------------|-----------|------------------------------|-----------------|
| 1 | **Attribute‑driven semantic units** (Attributes as Semantic Units chapter) | Visual concepts (attributes) are mined from language resources (e.g., WordNet, caption corpora) and used as semantic pivots between image features and natural language. | Attributes form a lightweight KG (attribute ↔ concept graph). Textual features (e.g., word embeddings) are aligned to the same attributes, enabling cross‑modal retrieval and captioning. | Image‑to‑text generation, visual grounding, VQA. |
| 2 | **Scene Description Graphs (SDGs)** | Each image is parsed into a graph where nodes are objects, attributes, and relations; edges encode spatial/semantic relations. | The SDG is enriched by two external knowledge sources:<br>• A *commonsense* KB derived from image captions (NLP‑extracted predicates).<br>• WordNet lexical ontology (synset relations). The graph thus merges visual detections, commonsense inference, and lexical semantics. | Automatic image captioning, image‑sentence alignment, visual storytelling. |
| 3 | **Graph‑enhanced neural summarization** (Structured Neural Summarization) | A conventional sequence encoder (e.g., Transformer) is augmented with a graph encoder that operates on a *dependency / discourse graph* extracted from the input text. | The graph captures long‑range dependencies that are hard to learn from the sequence alone. Textual tokens are first embedded, then passed through the graph neural network (GNN), and finally merged back into the sequence decoder. | Summarizing long documents, abstractive summarization. |
| 4 | **Retrieval‑based QA with triple‑convolutional networks** (TCNN & ATCNN) | The system retrieves a set of candidate knowledge entries from a QA KB, then reranks them by semantic matching. | Knowledge entries are first embedded as *triples* (subject–predicate–object). TCNN convolutions capture local patterns over these triples, while attention layers align the query text to the triple embeddings. | E‑commerce QA, customer support chatbots. |
| 5 | **Graph‑aware language models (not in the given titles but widely studied)** | Pre‑train language models on KG triples (e.g., ERNIE, K-BERT) or use KG embeddings as auxiliary signals. | KG embeddings are injected into the transformer layers or used to bias attention. | Question answering, entity linking, relation extraction. |
| 6 | **Joint KG–text representation learning** | Learn a shared vector space for KG entities and textual contexts (e.g., via co‑training). | Text passages containing entity mentions are aligned with KG nodes; a shared encoder (e.g., BERT) is fine‑tuned on both types of data. | Entity disambiguation, KG completion. |

---

### Common patterns in these methods

1. **Graph construction**  
   * From visual detections (SDG) → node = object/attribute, edge = spatial/semantic relation.  
   * From language corpora → nodes = entities/words, edges = dependency or co‑occurrence.  
   * From structured KBs (e.g., Freebase) → raw RDF triples.

2. **Feature fusion**  
   * **Early fusion** – concatenate raw text embeddings with KG embeddings before the encoder.  
   * **Late fusion** – process text and KG separately and combine scores (e.g., reranking).  
   * **Mid‑level fusion** – use GNNs to refine textual representations guided by graph structure.

3. **Inference or reasoning**  
   * Use graph propagation (GNN, message passing) to capture multi‑hop relationships.  
   * Apply rule‑based or probabilistic inference on the KG to enrich textual outputs (e.g., commonsense reasoning in SDG).

4. **Evaluation**  
   * End‑to‑end tasks: image captioning, VQA, summarization, QA.  
   * Alignment metrics: BLEU, METEOR for captions; ROUGE for summaries; precision/recall for retrieval.

---

### Take‑away

Approaches that merge knowledge graphs and text typically build a graph that captures semantic or relational structure (from vision, language, or curated KBs) and then apply neural or graph‑based methods to fuse this structured knowledge with unstructured textual signals. The fusion can be performed at multiple stages of the pipeline, enabling richer representations that improve performance on downstream tasks such as image captioning, question answering, and summarization."""

In [85]:
prompt = format_eval_prompt(query, context, answer)

messages = [
        {
            'role': 'user',
            'content': format_eval_prompt(query, context, answer),
        },
    ]

raw =  client.chat(model, messages=messages, stream=False)
print(raw.message.content)

{
  "context_relevance": 5,
  "answer_relevance": 5,
  "groundedness": 4,
  "justification": {
    "context": "The answer discusses exactly the papers and concepts present in the retrieved context (attributes, SDGs, structured summarization, TCNN). It aligns the answer’s content with the titles and summaries provided.",
    "answer": "The answer directly addresses the query by summarizing various approaches that combine knowledge graphs with text, matching the intent of the question.",
    "grounding": "The answer cites the core ideas from each paper in the context, but also adds additional related methods (e.g., graph‑aware language models, joint KG–text representation learning) that were not explicitly present in the retrieved snippets."
  }
}


In [89]:
result = json.loads(raw.message.content)